In [ ]:
import os #para manejar operaciones del sistema de archivos.
import time #para manejar retrasos temporales.
import requests #para realizar solicitudes HTTP.
from requests.exceptions import RequestException #Para manejar excepciones de la biblioteca 'requests'.
#para controlar el navegador y localizar elementos en la página.
from selenium import webdriver 
from selenium.webdriver.common.by import By
#para instalar y gestionar el driver de Chrome.
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
#para manejar archivos '.zip'
from zipfile import ZipFile

# Directorio para guardar los archivos descargados
download_dir = "C:\\Users\\diego\\Desktop\\BI_project\\Congreso_diputados" 
extract_dir = os.path.join(download_dir, "extracted") #define el directorio donde se extraerán los archivos contenidos 
#en los '.zip'

#crea estos directorios si no existen:
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Configurar el navegador
options = webdriver.ChromeOptions() #para configurar opciones de Chrome.
prefs = {'download.default_directory': download_dir} #establece la configuración del directorio de descarga 
# por defecto para Chrome
options.add_experimental_option('prefs', prefs) #añade las preferencias de descarga al navegador.

# Iniciar el navegador
# inicia una instancia del navegador Chrome con las opciones configuradas.
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
#define la URL de la página web de donde se descargarán los archivos.
url = "https://infoelectoral.interior.gob.es/es/elecciones-celebradas/area-de-descargas/"

# Abrir la página
# navega a la URL especificada.
driver.get(url)
# espera 5 segundos para asegurar que la página se cargue completamente.
time.sleep(5)  # Esperar a que se cargue la página

# Encontrar todos los enlaces que terminan en .zip y empiezan con 02_ pero no con PROV_02_
# encuentra todos los elementos <a> que contienen .zip y 02_ en su atributo href.
links = driver.find_elements(By.XPATH, "//a[contains(@href, '.zip') and contains(@href, '02_')]")
#filtra esos enlaces para asegurarse de que el nombre del archivo (última parte de la URL) empieza con 02_.
zip_links = [link.get_attribute('href') for link in links if link.get_attribute('href').split('/')[-1].startswith('02_')]

# Descargar y extraer los archivos
for zip_url in zip_links:
    file_name = os.path.join(download_dir, os.path.basename(zip_url))
    
    try:
        with requests.get(zip_url, stream=True) as r:
            r.raise_for_status()
            with open(file_name, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"Downloaded: {file_name}")
        
        # Extraer el archivo .zip
        with ZipFile(file_name, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted: {file_name}")
        
    except RequestException as e:
        print(f"Failed to download {zip_url}: {e}")

"""
- file_name define la ruta completa donde se guardará el archivo descargado.
- requests.get(zip_url, stream=True) envía una solicitud GET para descargar el archivo.
- r.raise_for_status() lanza una excepción si ocurre algún error durante la solicitud.
- open(file_name, 'wb') abre el archivo para escritura en modo binario.
- r.iter_content(chunk_size=8192) descarga el contenido del archivo en partes (chunks) de 8192 bytes.
- f.write(chunk) escribe cada parte en el archivo.
- print(f"Downloaded: {file_name}") imprime un mensaje indicando que el archivo se ha descargado.
- ZipFile(file_name, 'r') abre el archivo .zip para lectura.
- zip_ref.extractall(extract_dir) extrae todos los contenidos del archivo .zip en el directorio extract_dir.
- print(f"Extracted: {file_name}") imprime un mensaje indicando que el archivo se ha extraído.
- except RequestException as e maneja cualquier excepción de la biblioteca requests y imprime un mensaje de error."""

driver.quit()
print("All files downloaded and extracted successfully.")